In [1]:
! ./.g


[master 01b461f] web
 1 file changed, 2373 insertions(+), 1017 deletions(-)
 rewrite guardian comments scraper.ipynb (61%)
remote: Support for password authentication was removed on August 13, 2021. Please use a personal access token instead.
remote: Please see https://github.blog/2020-12-15-token-authentication-requirements-for-git-operations/ for more information.
fatal: unable to access 'https://github.com/macbuse/GUARDIAN.git/': The requested URL returned error: 403


In [59]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [60]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','a') as fp:
    fp.write('\n'.join(lks))

In [1]:
import re 
import requests
import pickle
import json

In [9]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/21/joe-biden-democrat-convention-nice-guy-trump-fight'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/23/the-cruel-exams-algorithm-has-laid-bare-the-unfairness-at-the-heart-of-our-schools'
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/01/us-mainstream-right-vigilante-terror'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/31/politicians-covid-19-statistics-statisticians'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'
article_url = 'https://www.theguardian.com/commentisfree/2021/may/10/retail-politics-movements-polarising-projects-promise'

In [2]:
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/29/rival-scientists-lockdowns-scientific-covid-19'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/08/johnson-and-sturgeon-will-need-a-better-philosophy-than-precaution-to-rid-us-of-covid'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/10/continual-local-lockdowns-answer-covid-control'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/11/the-rebel-scientists-cause-would-be-more-persuasive-if-it-werent-so-half-baked'

article_url = 'https://www.theguardian.com/commentisfree/2021/apr/03/undermining-the-astrazeneca-jab-is-a-dangerous-act-of-political-folly#comment-148440261'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/09/boris-johnson-brexit-belfast-violence-eu-good-friday-agreement'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/02/culture-war-government-race-report'
article_url = 'https://www.theguardian.com/commentisfree/2021/mar/18/britain-sunny-vaccine-optimism-april-delivery-delay'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/15/britain-harness-social-sciences-covid-pandemic-deprivation'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/16/government-race-report-evidence#comment-148662066'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/20/britain-falling-birthrate-covid-pandemic-conservatives-removed-support-for-parents'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/21/bmi-eating-disorder-pandemic-treatment'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/30/government-delays-covid-public-inquiry-health-crisis'
article_url = 'https://www.theguardian.com/commentisfree/2021/may/07/humiliation-hartlepool-smalltown-detective-di-starmer'
article_url ='https://www.theguardian.com/commentisfree/2021/aug/06/uk-government-experts-vaccinating-under-16s'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/01/there-is-a-case-for-vaccine-passports-but-ministers-are-failing-to-make-it'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/09/america-is-flying-blind-when-it-comes-to-the-delta-variant'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/10/covid-vaccines-britain-scientific-solution-jabs'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/13/denial-anti-vaxxers-climate-sceptics'
article_url = 'https://www.theguardian.com/commentisfree/2018/apr/11/bbc-andrew-neil-media-politics'

article_url = 'https://www.theguardian.com/commentisfree/2021/aug/15/vaccine-passports-are-less-a-threat-to-liberty-than-a-mark-of-solidarity'
article_url = 'https://www.theguardian.com/theobserver/commentisfree/2021/aug/14/will-we-reach-herd-immunity-for-the-new-coronavirus'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/15/vaccine-hesitancy-broken-relationship-state-conspiracy-theorists'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/17/dominic-raab-news-foreign-secretary-taliban-kabul'
article_url = 'https://www.theguardian.com/commentisfree/2021/dec/11/so-which-of-these-politicians-is-a-neoliberal-not-one-of-them'

In [ ]:


key = my_keys.keys['guardian']

In [3]:
#scrape the page and get the short url
# 15/8/21  this failed and gave /p/temp
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)
uu = r.text
print(short_url)

#the short url allows us to access the discussion
# 15/8/21 api.nextgen.guardianapps.co.uk/discussion/ failed on 
# https://discussion.theguardian.com/discussion-api/discussion/p/tfmxg
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn,'wb') as fp:
    pickle.dump(json_data,fp)

/p/kv6n2
getting comments page no. 1
there are 12 pages
getting comments page no. 2
getting comments page no. 3
getting comments page no. 4
getting comments page no. 5
getting comments page no. 6
getting comments page no. 7
getting comments page no. 8
getting comments page no. 9
getting comments page no. 10
getting comments page no. 11
getting comments page no. 12
archiving to kv6n2.pkl


In [3]:
ls


 8djv8.pkl        gn7h4.pkl                            links_2020_8.txt
 94f6b.pkl        goose_test_editorialis_miner.ipynb   links.txt
 afuahirsch.csv  'guardian api and comments.ipynb'     matplotlib.ipynb
 edt72.pkl        guardian_api_prod.ipynb              None0000000.png
 eghyd.pkl       'guardian comments scraper.ipynb'     owen-jones.csv
 egqhf.pkl        guardian_extractor.ipynb             quad_forms.ipynb
 ej7f5.pkl        guardian_nlp.ipynb                   README.md
 ejdxt.pkl        guardian_photo.ipynb                 tbca8.pkl
 et8zg.pkl        h2tzt.pkl                            tdvxp.pkl
 etenk.pkl        h44zn.pkl                            te5tn.pkl
 eyhx8.pkl        h452h.pkl                            tedzk.pkl
 eytkx.pkl        h4cq2.pkl                            tfmxg.pkl
 f3ke9.pkl        h5j4n.pkl                            tfy7d.pkl
 f4a3q.pkl        h6v39.pkl                            tgaxq.pkl
 f4vfe.pkl        h8qmd.pkl                            tgq

In [8]:
json_data = pickle.load(open('tgqq8.pkl','rb'))


## remember each page of comments has been stored separately

In [9]:
dd = json.loads(json_data[0])
len(json_data)

31

In [4]:
xx = json.loads(json_data[0])
xx.keys()

dict_keys(['postedCommentHtml', 'commentsHtml', 'commentCount', 'refreshStatus', 'lastPage', 'paginationHtml'])

In [136]:
print( xx['commentsHtml'][200:700])

 class="d-comment
            d-comment--top-level"
           id="comment-144422422"
           data-comment-id="144422422"
           data-comment-author="Hushing"
           data-comment-replies="0"
           data-comment-timestamp="2020-10-11T14:25:06.000+01:00"
           data-comment-author-id="12407494"
           data-comment-highlighted="false"
           itemscope itemtype="http://schema.org/Comment">

    <div class="d-comment__inner d-comment__inner--top-level">
        <div class="


In [16]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')
pcid = re.compile('\sid="comment-(.*?)"')
pp = re.compile('<li class="d-comment(.*?)>',re.DOTALL)

auths = []
md = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh), 
                           pid.findall(hh)
                          )))
    md.extend(pp.findall(hh))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [102]:
#print( '\n'.join([x for x in md if 'pinkie' in x.lower()]))

https://discussion.theguardian.com/comment-permalink/144375739

In [8]:
auths = dict(auths)

In [9]:
comments_url

'https://api.nextgen.guardianapps.co.uk/discussion//p/hvpq8.json'

In [5]:

from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    
    auth = [ x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    #this was a pain to do really
    #the problem is blocked comments
    comment_id = [ x['id'][len('comment-'):] for x in soup.findAll('li')]
 
    
    ss = '##{}\nhttps://discussion.theguardian.com/comment-permalink/{}\n{}\n\n{}'
    return [ ss.format(a,b,d,c.strip())  
                   for a,b,d,c in zip(auth, comment_id, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]


from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

# Random data hacking

In [6]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##BOBBYLEOPARD', 30),
 ('##MAXIBOY339', 20),
 ('##CHICKENSANDGOATS', 13),
 ('##MONKEY_HANGER', 12),
 ('##JASONBRADLEY', 11),
 ('##WHILEROMEBURNS', 11),
 ('##PARCELOFROGUE', 10),
 ('##NEUTRALPADDY', 10),
 ('##BARCIAD', 9),
 ('##3RDROCK', 9),
 ('##ZIONTRAIN', 8),
 ('##DREAMER06', 7),
 ('##TOMPER2', 7),
 ('##ONEEYEDMANISKING', 7),
 ('##LISAMARIE3', 7),
 ('##PINKROOM', 7),
 ('##RICOS321', 7),
 ('##ALPO88', 6),
 ('##IHATECAPTCHAS', 6),
 ('##STILLBREATHINGJUST', 6),
 ('##FALLOWFIELD', 6),
 ('##GEGENBEISPIEL', 6),
 ('##MICK233489', 6),
 ('##HAGGALA', 6),
 ('##HARRYTHEHAWK', 6),
 ('##MFISHER', 6),
 ('##FATBOY2001', 5),
 ('##EKNGEWNGEJWGNGE', 5),
 ('##BLISSEX', 5),
 ('##FAIRSHARES', 5),
 ('##CHRISWR', 5),
 ('##ROBBO100', 5),
 ('##KOFIANNAN', 5),
 ('##A_REBOUR', 5)]

In [ ]:
! ./.g


In [7]:
print(all_comments[0])

##SUPERSTRIKER
https://discussion.theguardian.com/comment-permalink/153639318
https://profile.theguardian.com/user/id/104468511

I think we all know what a neo liberal is those opposed to government intervention, on economic or broad social grounds. Thatcher and Reagan fall into that category, even if few of their policies where neo liberal. They set the ball rolling for the neo liberals who came after them. Thatcher sold off the council houses that neo liberals policies use to drive millions into the yokes for the rentier class to live of their backs. Private rent tenants have their noses held to the grindwheel lest they fall into arrears and eviction.
Tony Blair? Not so much. Cameron? A fucking halfwit, but a full frontal attack on the welfare state saw the rise of food banks, homelessness and a general hatred of those ousted out of the labour market. Sure tools to drive the neo liberal agenda.
The fact that some I'll informed people use the term as a catchall insult in the same way 

In [8]:
all_comments.sort(key=lambda x : -len(x) )
print( '\n\n'.join(all_comments[:20]) )

##ANDYFOSTER353
https://discussion.theguardian.com/comment-permalink/153639159
https://profile.theguardian.com/user/id/14132951

The problem with Nick Cohen's writing here is a lack of logic, produced by a lack of courage. On the one hand:
"Like so much insider jargon, neoliberal is a pathetically weak term. To take an everyday example: since Thatcher’s government privatised the water industry in 1989, managers and shareholders have exploited their monopoly power to take almost £60bn in profit. In 2020, the companies spent 3.1m hours dumping sewage into rivers. So great has been their failure to reinvest even a small portion of the money they take, south-east England may soon face water shortages."
Dead right. But then:
"Only moderate centre-left parties have won elections in western countries and their victories are rare enough. If you insist that they are as much a part of a neoliberal conspiracy as the right, then there is no point in fighting to remove the right from power."
So Nic

In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
#nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [12]:
target = 'decolon'
target = 'science'
target = 'trump'


xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )

#print( '\n\n'.join(xx).replace(target, ) )
pxx = re.compile(target, re.I)
print( pxx.sub('**' + target.upper() + '**', 
              '\n\n'.join(xx)) )
               
#print( '\n\n'.join(xx) )


##SNAFU5001
https://discussion.theguardian.com/comment-permalink/151314550
https://profile.theguardian.com/user/id/11745420

Ah bless, a dispirited Fox News ‘journalist’ justifying their existence.
 Nope not a conservative or a liberal just a realist. 

So tell us Snafu5001 how does injecting bleach cure Covid or protect us from Covid? Because Fox News certainly didn’t bother holding **TRUMP** to account on this one.
 Ideologs have a tendency to spin everything to support their ideology. Was **TRUMP** an asshole, yep. Does that make me liberal now? No. 

Secondly how did **TRUMP**s deal with the Taliban work out? The US was supposed to have left by May and Fox News **TRUMP**eted **TRUMP** super peace deal with the Taliban. Any news and comment from Snafu5001 on the reality of that?
 People said on both sides of the isle that his peace deal was stupid. Having spent time in Afghanistan I thought it was stupid. **TRUMP** went around changing all of Obama's deals and Biden and team have be

In [20]:
(26/31) / (74/69)

0.7820401046207497

In [64]:
23**2

529

## 

In [21]:
84 + 127

211

In [24]:
84/211/.31

1.2842073077511085

In [25]:
127/211/.69

0.8723126588364586

In [26]:
1.28/.87

1.471264367816092